https://github.com/pyproj4/pyproj  
https://pyproj4.github.io/pyproj/stable/  

In [ ]:
# conda install pyproj # 安装

In [ ]:
!conda list | grep pyproj

# CRS
A pythonic Coordinate Reference System manager.

In [ ]:
from pyproj import CRS

## 初始化

In [ ]:
crs = CRS.from_epsg(4326) # Make a CRS from an EPSG code

In [ ]:
crs

## 比较

In [ ]:
CRS.from_epsg(4326) == CRS.from_string("epsg:4326") == CRS.from_user_input(4326)

In [ ]:
CRS.from_epsg(4326).equals(CRS.from_string("epsg:4326"))

In [ ]:
CRS.from_epsg(4326).is_exact_same(CRS.from_string("epsg:4326"))

## 属性

In [ ]:
crs.list_authority()

In [ ]:
crs.name

### 时区

In [ ]:
CRS.from_string("epsg:4326").utm_zone

In [ ]:
CRS.from_string("epsg:32649").utm_zone

### 边界

In [ ]:
crs.area_of_use

In [ ]:
crs.area_of_use.name

In [ ]:
crs.area_of_use.bounds # 输入参数的范围

In [ ]:
crs.area_of_use.east

In [ ]:
crs.area_of_use.west

In [ ]:
crs.area_of_use.south

In [ ]:
crs.area_of_use.north

### 椭球

In [ ]:
crs.ellipsoid # 椭球体

In [ ]:
print("椭球长半轴 =", crs.ellipsoid.semi_major_metre, ", 椭球短半轴 =", crs.ellipsoid.semi_minor_metre, ", 1/扁率 =", crs.ellipsoid.inverse_flattening)

### 坐标系

In [ ]:
crs.coordinate_system # 坐标系统

In [ ]:
crs.axis_info # 坐标轴

### 本初子午线

In [ ]:
crs.prime_meridian

In [ ]:
crs.prime_meridian.unit_name

In [ ]:
crs.prime_meridian.unit_conversion_factor

In [ ]:
crs.prime_meridian.longitude

### 基准

In [ ]:
crs.datum

## geocentric, geographic, projected  
refer to: https://blog.csdn.net/wuwuku123/article/details/104711203

In [ ]:
CRS.from_string("epsg:4326").type_name

In [ ]:
CRS.from_string("epsg:32649").type_name

In [ ]:
crs.is_geocentric # 地心坐标系

In [ ]:
crs.is_geographic # 大地坐标系

In [ ]:
crs.is_projected # 投影坐标系

In [ ]:
CRS.from_epsg(4326).geodetic_crs

In [ ]:
CRS.from_epsg(32649).geodetic_crs

In [ ]:
CRS.from_epsg(4326).source_crs

In [ ]:
CRS.from_epsg(32649).source_crs

In [ ]:
crs1 = CRS.from_epsg(32649).geodetic_crs
crs2 = CRS.from_epsg(32649).source_crs
crs1.is_exact_same(crs2)

### **<font color="red">Retrieve the geodetic CRS based on original CRS</font>**
<font color="red">根据投影CRS, 反推大地CRS</font>

In [ ]:
from pyproj import CRS
crs_utm = CRS.from_epsg(32649)
crs_wgs84 = crs_utm.geodetic_crs
crs_wgs84.to_epsg()

## 导出

In [ ]:
CRS.from_string("epsg:4326").to_epsg()

In [ ]:
CRS.from_string("epsg:4326").to_authority()

In [ ]:
CRS.from_string("epsg:4326").to_string()

In [ ]:
CRS.from_string("epsg:4326").to_dict()

In [ ]:
CRS.from_string("epsg:32649").to_dict()

In [ ]:
CRS.from_string("epsg:4326").to_proj4()

In [ ]:
CRS.from_string("epsg:32649").to_proj4()

In [ ]:
CRS.from_string("epsg:4326").to_json()
CRS.from_string("epsg:4326").to_json_dict()
CRS.from_string("epsg:4326").to_wkt()

# **<font color="red">Find UTM CRS by Latitude and Longitude</font>**
根据经纬度, 推导UTM投影CRS

In [ ]:
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

## example

In [ ]:
lon = 113.595417400
lat = 22.744435950

utm_crs_list = query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest=AreaOfInterest(
        west_lon_degree=lon,
        south_lat_degree=lat,
        east_lon_degree=lon,
        north_lat_degree=lat,
    ),
)


In [ ]:
len(utm_crs_list)

In [ ]:
for utm_crs in utm_crs_list:
    crs_utm = CRS.from_epsg(utm_crs.code)
    print(crs_utm)

# Transformations from one CRS to another CRS

In [ ]:
from pyproj import CRS
from pyproj import Transformer

Note that crs_wgs84 has the latitude (north) axis first and the crs_utm has the easting axis first. This means that in the transformation, we will need to input the data with latitude first and longitude second. Also, note that the second projection is a UTM projection with bounds (-84.0, 23.81, -78.0, 84.0) which are in the form (min_x, min_y, max_x, max_y), so the transformation input/output should be within those bounds for best results.  
请注意，crs_wgs84 首先具有纬度（北）轴，而 crs_utm 首先具有东向轴。 这意味着在转换中，我们需要先输入纬度，然后再输入经度。 另外，请注意，第二个投影是具有边界 (-84.0, 23.81, -78.0, 84.0) 的 UTM 投影，其形式为 (min_x, min_y, max_x, max_y)，因此转换输入/输出应在这些范围内 以获得最佳效果。

In [ ]:
crs_wgs84 = CRS.from_epsg(4326)
crs_utm = CRS.from_epsg(32649)
transformer = Transformer.from_crs(crs_wgs84, crs_utm)

In [ ]:
transformer

## 比较

In [ ]:
transformer.is_exact_same(Transformer.from_crs(4326, 32649))

In [ ]:
transformer.is_exact_same(Transformer.from_crs("EPSG:4326", "EPSG:32649"))

## 属性

In [ ]:
type(transformer)

In [ ]:
transformer.name

In [ ]:
transformer.definition

In [ ]:
transformer.description

In [ ]:
transformer.accuracy

In [ ]:
transformer.area_of_use

In [ ]:
transformer.is_network_enabled

In [ ]:
transformer.source_crs

In [ ]:
transformer.target_crs

In [ ]:
transformer.transform_bounds

## **<font color="red">转换/transform</font>**

In [ ]:
import math
lon = 113.60
lat = 22.74

lons = [113.60, 114.60, 115.60]
lats = [22.74, 23.74, 24.74]

points = [(22.74, 113.60), (23.74, 114.60), (24.74, 115.60)] #lat, lon

单个用法1

In [ ]:
transformer = Transformer.from_crs(crs_wgs84, crs_utm)
transformer.transform(lat, lon)

单个用法2

In [ ]:
transformer.transform(lat/180*math.pi, lon/180*math.pi, radians=True) # 弧度

批量用法1

In [ ]:
transformer.transform(lats, lons)

批量用法2

In [ ]:
import pyproj
for x, y in transformer.itransform(points): # 批量转换
    print("utm:\t", x, y)
    if transformer.has_inverse:
        lat_, lon_ = transformer.transform(x, y, direction=pyproj.enums.TransformDirection.INVERSE) # 逆向映射
        print("wgs84:\t", lon_, lat_)

单个用法3

In [ ]:
transformer = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True) # 仅交换输入的顺序，输出的顺序不变
transformer.transform(lon, lat)

## 导出

In [ ]:
transformer = Transformer.from_crs(crs_wgs84, crs_utm)

In [ ]:
transformer.to_json()

In [ ]:
transformer.to_json_dict()

In [ ]:
transformer.to_proj4()

In [ ]:
transformer.to_wkt()

# Geodesic calculations

## Creating Geod class 

In [ ]:
from pyproj import CRS, Geod

In [ ]:
geod = CRS("epsg:4326").get_geod()

In [ ]:
geod

In [ ]:
geod = Geod(ellps="WGS84") # using an ellipsoid name

In [ ]:
geod

In [ ]:
CRS("epsg:4326").get_geod() == Geod('+a=6378137 +f=0.0033528106647475126')

In [ ]:
CRS("epsg:4326").get_geod() == Geod(ellps="WGS84")

## Geodesic line length

In [ ]:
lons = [-74, -102, -102, -131, -163, 163, 172, 140, 113, 88, 59, 25, -4, -14, -33, -46, -61]
lats = [-72.9, -71.9, -74.9, -74.3, -77.5, -77.4, -71.7, -65.9, -65.7, -66.6, -66.9, -69.8, -70.0, -71.0, -77.3, -77.9, -74.7]

In [ ]:
geod = CRS("epsg:4326").get_geod()
length = geod.line_length(lons, lats)
print("length =", length, "m")

In [ ]:
geod.line_lengths(lons, lats)

In [ ]:
len(lons) - len(geod.line_lengths(lons, lats))

In [ ]:
sum(geod.line_lengths(lons, lats))-length

## Geodesic area

In [ ]:
geod = CRS("epsg:4326").get_geod()
poly_area, poly_perimeter = geod.polygon_area_perimeter(lons, lats)
print("面积 =", poly_area, "| 周长 =", poly_perimeter)

## npts
Given a single initial point and terminus point, returns a list of longitude/latitude pairs describing npts equally spaced intermediate points along the geodesic between the initial and terminus points.  
给定一个初始点和终点，返回一个经度/纬度对列表，描述沿初始点和终点之间的测地线等间距的 npts 中间点。

In [ ]:
boston_lat = 42.+(15./60.); boston_lon = -71.-(7./60.)
portland_lat = 45.+(31./60.); portland_lon = -123.-(41./60.)
# find ten equally spaced points between Boston and Portland.
lonlats = geod.npts(boston_lon,boston_lat,portland_lon,portland_lat,10)
for lon,lat in lonlats:
    print(lon, lat)

In [ ]:
import math
dg2rad = math.radians(1.)
rad2dg = math.degrees(1.)
lonlats = geod.npts(
   dg2rad*boston_lon,
   dg2rad*boston_lat,
   dg2rad*portland_lon,
   dg2rad*portland_lat,
   10,
   radians=True
)
for lon,lat in lonlats:
    print(rad2dg*lon, rad2dg*lat)

# other

In [ ]:
import pyproj
pyproj.datadir.get_data_dir() # get data directory